# Dive into GeoParquet using Python (GeoPandas + PyArrow)

While this notebook focuses on exploring GeoParquet files using GeoPandas and PyArrow, note that there are many alternatives - also in Python (DuckDB, geoarrow-rs, Apache Sedona, ..) - to work with Parquet files.

## Example: the Overture building data for Delft

Overture is a collaborative open-data initiative focusing on open map data (https://overturemaps.org/). It has data sets about buildings, streets, addresses, boundaries, land use, etc, ànd makes it data available as GeoParquet files on S3 (https://overturemaps.org/blog/2025/why-we-chose-geoparquet-breaking-down-data-silos-at-overture-maps/).

Explore the data interactively at https://explore.overturemaps.org/#16.08/52.007122/4.366463 (which also allows to download a small subset).

As an example, let's explore the building data (~250 GB in total). Using GeoPandas, we can read a small subset of the data for an area around the center of Delft by providing a bounding box:

In [1]:
import geopandas

In [2]:
%%time
gdf = geopandas.read_parquet(
    "s3://overturemaps-us-west-2/release/2025-10-22.0/theme=buildings/type=building/",
    bbox=(4.29, 51.98, 4.38, 52.03)
)

CPU times: user 4.99 s, sys: 1.33 s, total: 6.32 s
Wall time: 34.7 s


In [3]:
gdf[["id", "geometry", "subtype"]]

,id,geometry,subtype
0,263563f4-65f3-42bc-bf75-25f7f9b7b10b,"POLYGON ((4.38001 52.02984, 4.37997 52.02988, ...",residential
1,31da39f0-e9e0-4fad-8b3c-26fd845e3061,"POLYGON ((4.37954 52.02974, 4.3795 52.02973, 4...",None
2,dcf4ca67-4b47-4508-b629-c5241a3696b6,"POLYGON ((4.37976 52.02973, 4.37972 52.02977, ...",residential
3,18e08477-9c02-450c-ac8b-7d4893d5d161,"POLYGON ((4.37974 52.02961, 4.37966 52.02958, ...",residential
4,40bc7f22-5fcd-44cd-b3a0-99fedcf5b152,"POLYGON ((4.37966 52.02928, 4.37956 52.02937, ...",None
...,...,...,...
57316,2219c574-7cfc-4d99-9b41-9c75a9816f9b,"POLYGON ((4.29019 52.02958, 4.29012 52.02968, ...",residential
57317,f3e2fefb-2068-453e-b477-ec67f0ba7f47,"POLYGON ((4.29012 52.02955, 4.29005 52.02964, ...",residential
57318,d1270d62-0ed8-426e-b6c2-140b31fb6bb0,"POLYGON ((4.29004 52.02975, 4.29002 52.02978, ...",outbuilding
57319,96148295-6738-4f38-9627-368fefff92ab,"POLYGON ((4.29001 52.02984, 4.28998 52.02989, ...",residential


The subset of data is also provided in the workshop material (downloaded using the [overturemaps CLI](https://github.com/OvertureMaps/overturemaps-py) with `overturemaps download -o delft.parquet -f geoparquet -t building --bbox=4.29,51.98,4.38,52.03`), and can be read as:

```python
gdf = geopandas.read_parquet("delft.parquet")
```

Let's visualize the downloaded data quickly to verify the result (we can also use the built-in `gdf.plot()`, but lonboard gives an easy interactive map that works for larger data):

In [4]:
import lonboard

In [5]:
lonboard.viz(gdf)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

## How does this GeoParquet dataset look like?

Let's explore the details of this dataset (the different files, the schema of the files, etc). Here, we use the `pyarrow.dataset` functionality to interact with a multi-file dataset:

In [6]:
import pyarrow.dataset as ds

In [7]:
dataset = ds.dataset("s3://overturemaps-us-west-2/release/2025-10-22.0/theme=buildings/type=building/")

Schema of the Parquet dataset (this specific dataset has lots of additional attribute columns, but notice the "geometry" binary columns and the "bbox" struct column at the start of the schema):

In [8]:
dataset.schema

id: string
geometry: binary
bbox: struct<xmin: float, xmax: float, ymin: float, ymax: float>
  child 0, xmin: float
  child 1, xmax: float
  child 2, ymin: float
  child 3, ymax: float
version: int32
sources: list<element: struct<property: string, dataset: string, license: string, record_id: string, update_t (... 65 chars omitted)
  child 0, element: struct<property: string, dataset: string, license: string, record_id: string, update_time: string, c (... 50 chars omitted)
      child 0, property: string
      child 1, dataset: string
      child 2, license: string
      child 3, record_id: string
      child 4, update_time: string
      child 5, confidence: double
      child 6, between: list<element: double>
          child 0, element: double
level: int32
subtype: string
class: string
height: double
names: struct<primary: string, common: map<string, string ('common')>, rules: list<element: struct<variant: (... 159 chars omitted)
  child 0, primary: string
  child 1, common: map<string

The "geometry" column is storing the actual geometries (as WKB data), and the "bbox" column helps us with reading a subset of the data more efficiently (more details on that later).

Here, the partitioned dataset consists of 236 files:

In [9]:
len(dataset.files)

236

Each of those files is around a GB, giving a total size of the dataset of around 250 GB:

In [10]:
import pyarrow.fs

In [11]:
fs, path = pyarrow.fs.FileSystem.from_uri("s3://overturemaps-us-west-2/release/2025-10-22.0/theme=buildings/type=building/")

In [12]:
files = fs.get_file_info(pyarrow.fs.FileSelector(path, recursive=True))

In [13]:
sum([f.size for f in files]) / 1000**3

253.504071608

And the total number of records:

In [14]:
dataset.count_rows()

2540818466

GeoParquet files are in practice "just" normal Parquet files, but with some additional custom metadata about the geometry column under the "geo" key.

In [15]:
import pyarrow.parquet as pq

In [16]:
file_metadata = pq.read_metadata(dataset.files[0], filesystem=dataset.filesystem)
file_metadata

  created_by: parquet-mr version 1.13.1 (build 70ff76ac5d9a2592713433742d073af4ca18169a)
  num_columns: 42
  num_rows: 11047341
  num_row_groups: 71
  format_version: 1.0
  serialized_size: 404966

In [17]:
file_metadata.metadata.keys()

dict_keys([b'org.apache.spark.sql.parquet.row.metadata', b'geo', b'org.apache.spark.version'])

In [18]:
import json

json.loads(file_metadata.metadata[b"geo"])

{'version': '1.1.0',
 'primary_column': 'geometry',
 'columns': {'geometry': {'encoding': 'WKB',
   'geometry_types': ['Polygon', 'MultiPolygon'],
   'bbox': [-179.9685336, -84.2945957, -2.8229824, -22.499915],
   'covering': {'bbox': {'xmin': ['bbox', 'xmin'],
     'ymin': ['bbox', 'ymin'],
     'xmax': ['bbox', 'xmax'],
     'ymax': ['bbox', 'ymax']}}}}}

This metadata indicates that the column with name "geometry" is a geometry column using the WKB encoding to store the geometries.

Information about the Coordinates Reference System (CRS) is missing here, which means that the default of OGC:WGS84 applies (geographic longitude/latitude coordinates).

## Examples of other tools

- QGIS:
  - https://medium.com/radiant-earth-insights/a-deep-dive-into-geoparquet-downloader-qgis-plug-in-017c0b1facb1
  - If QGIS is installed with GDAL built with Parquet support, it can open GeoParquet files
- DuckDB: https://docs.overturemaps.org/getting-data/duckdb/
- GDAL cli

In [19]:
!gdal vector info --format=text /vsis3/overturemaps-us-west-2/release/2025-10-22.0/theme=buildings/type=building/ 

INFO: Open of `/vsis3/overturemaps-us-west-2/release/2025-10-22.0/theme=buildings/type=building/'
      using driver `Parquet' successful.

Layer name: type=building
Geometry: Multi Polygon
Feature Count: 2540818466
Extent: (-179.999997, -84.294596) - (179.997291, 83.094002)
Layer SRS WKT:
GEOGCRS["WGS 84",
    ENSEMBLE["World Geodetic System 1984 ensemble",
        MEMBER["World Geodetic System 1984 (Transit)"],
        MEMBER["World Geodetic System 1984 (G730)"],
        MEMBER["World Geodetic System 1984 (G873)"],
        MEMBER["World Geodetic System 1984 (G1150)"],
        MEMBER["World Geodetic System 1984 (G1674)"],
        MEMBER["World Geodetic System 1984 (G1762)"],
        MEMBER["World Geodetic System 1984 (G2139)"],
        MEMBER["World Geodetic System 1984 (G2296)"],
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ENSEMBLEACCURACY[2.0]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipso

## Spatial index versus spatial partitioning

> From https://cloudnativegeo.org/blog/2024/12/interview-with-kyle-barron-on-geoarrow-and-geoparquet-and-the-future-of-geospatial-data-analysis/

Spatial **partitioning** in GeoParquet breaks data into multiple files and "chunks" based on a spatial attribute like a country or quadkey, allowing systems to skip entire files and chunks for faster querying.  

Spatial **indexing**, on the other hand, uses a data structure (like an R-tree) within a file to efficiently locate individual features based on their position, which can improve performance but can make the file itself larger and doesn't scale as well to larger data. 

<img src="img/geoparquet_layout.png" style="width: 600px;"/>

Note! For GeoParquet 1.x files, filtering at the chunk level requires the `bbox` column to ensure the file contains statistics about min/max coordinate values per chunk.

By adjusting the chunk size, the writer can choose the tradeoff between index size and indexing efficiency.

---

Let's illustrate this difference by comparing some queries using the Delft buildings subset for GeoParquet (with spatially partitioned chunks) vs FlatGeoBuf (which does have an actual spatial index).

To make the differences more pronounced, let's download the full 1GB file that contains the data for Delft ([download link](https://overturemaps-us-west-2.s3.us-west-2.amazonaws.com/release/2025-10-22.0/theme=buildings/type=building/part-00141-c5e0b5f2-08ff-4192-af19-c572ecc088f1-c000.zstd.parquet), expand section below to see how to get this link), and convert that to FlatGeoBuf:

```bash
ogr2ogr part-00141.fgb part-00141-c5e0b5f2-08ff-4192-af19-c572ecc088f1-c000.zstd.parquet -nlt PROMOTE_TO_MULTI
```

<details>
<summary>(click here for the code to figure out download link for Overture buildings file containing Delft)</summary>

```python
import pandas as pd
import shapely

# fetching an overview of all individual Parquet files in the overture data
collections = pd.read_parquet("https://stac.overturemaps.org/2025-10-22.0/collections.parquet")
collections["geometry"] = geopandas.GeoSeries.from_wkb(collections["geometry"])
collections_building = collections[collections["collection"] == "building"]

# filter all files for which the bounding box intersects with Delft area
collections_delft = collections_building[shapely.intersects(collections_building.geometry, shapely.box(4.29, 51.98, 4.38, 52.03))]

# this gives two files, for which in this case we want the second file with index 332
delft_assets = collections_delft.loc[332, "assets"]
print(delft_assets["aws-https"]["href"])
# -> https://overturemaps-us-west-2.s3.us-west-2.amazonaws.com/release/2025-10-22.0/theme=buildings/type=building/part-00141-c5e0b5f2-08ff-4192-af19-c572ecc088f1-c000.zstd.parquet
```

</details>
<br>

If you don't have the full file, you can also use the provided subset for just the Delft area:

```python
gdf = geopandas.read_parquet("delft.parquet")
gdf.to_file("delft.fgb")
```


In [1]:
import geopandas

In [2]:
fpath_parquet = "part-00141-c5e0b5f2-08ff-4192-af19-c572ecc088f1-c000.zstd.parquet"  # or delft.parquet"
fpath_fbg = "part-00141.fgb"  # or "delft.fgb"

When querying a very small subset based on position, FlatGeoBuf is faster:

In [3]:
len(geopandas.read_file(fpath_fbg, bbox=(4.3300, 52.0010, 4.3315, 52.0015)))

26

In [4]:
%timeit geopandas.read_file(fpath_fbg, bbox=(4.3300, 52.0010, 4.3315, 52.0015))

1.65 ms ± 7.16 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [5]:
%timeit geopandas.read_file(fpath_parquet, bbox=(4.3300, 52.0010, 4.3315, 52.0015))

97.8 ms ± 2.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


When reading a larger subset (the original Delft bbox), the difference becomes less pronounced (here, we are specifying `use_arrow=True` to make use of the faster Arrow transport mechanism from GDAL to geopandas):

In [6]:
%timeit geopandas.read_file(fpath_fbg, bbox=(4.29, 51.98, 4.38, 52.03), use_arrow=True)

108 ms ± 2.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%timeit geopandas.read_file(fpath_parquet, bbox=(4.29, 51.98, 4.38, 52.03), use_arrow=True)

260 ms ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In contrast, when reading a single column, this is much faster with Parquet:

In [8]:
%timeit geopandas.read_file(fpath_fbg, columns=["height"], read_geometry=False, use_arrow=True)

2.19 s ± 45.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit geopandas.read_file(fpath_parquet, columns=["height"], read_geometry=False, use_arrow=True)

178 ms ± 21.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Also filtering on an attribute is in this case faster with Parquet (the resulting filter gives around 65,000 features, which is around 0.7% of the rows):

In [10]:
%timeit geopandas.read_file(fpath_fbg, use_arrow=True, where="subtype = 'commercial'")

7.22 s ± 217 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
import pyarrow.dataset as ds

In [12]:
%timeit geopandas.read_parquet(fpath_parquet, filters=ds.field("subtype") == "commercial")

1.25 s ± 15.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Exploring the spatial partitioning of the Overture buildings dataset

First, let's visualize the file-level spatial partitions.

For Overture, they actually provide an overview parquet file with information of all files:

In [1]:
import pandas as pd
import geopandas

collections = pd.read_parquet("https://stac.overturemaps.org/2025-10-22.0/collections.parquet")
collections["geometry"] = geopandas.GeoSeries.from_wkb(collections["geometry"])
collections_building = collections[collections["collection"] == "building"]
collections_building[["assets", "geometry", "num_row_groups", "num_rows"]]

,assets,geometry,num_row_groups,num_rows
191,{'aws-https': {'href': 'https://overturemaps-u...,"POLYGON ((-179.96853 -84.2946, -2.82298 -84.29...",71,11047341
192,{'aws-https': {'href': 'https://overturemaps-u...,"POLYGON ((-71.71882 -33.75032, -56.24995 -33.7...",67,10485376
193,{'aws-https': {'href': 'https://overturemaps-u...,"POLYGON ((-67.50004 -28.82788, -50.62491 -28.8...",80,12561972
194,{'aws-https': {'href': 'https://overturemaps-u...,"POLYGON ((-56.25007 -25.3127, -46.40526 -25.31...",77,11507573
195,{'aws-https': {'href': 'https://overturemaps-u...,"POLYGON ((-50.62511 -27.42209, -44.99992 -27.4...",67,10331158
...,...,...,...,...
422,{'aws-https': {'href': 'https://overturemaps-u...,"POLYGON ((28.12494 -29.5314, 45.00003 -29.5314...",78,11487363
423,{'aws-https': {'href': 'https://overturemaps-u...,"POLYGON ((22.49998 -33.75032, 31.22361 -33.750...",78,11080574
424,{'aws-https': {'href': 'https://overturemaps-u...,"POLYGON ((22.50003 -29.53126, 32.96367 -29.531...",75,11244133
425,{'aws-https': {'href': 'https://overturemaps-u...,"POLYGON ((28.12372 -26.71894, 33.75006 -26.718...",67,10425585


More in general, we can get the partition bounding box information from the metadata:

In [2]:
import json

import pyarrow.dataset as ds
import shapely

import lonboard

In [3]:
dataset = ds.dataset("s3://overturemaps-us-west-2/release/2025-10-22.0/theme=buildings/type=building/")

In [4]:
file_bounds = [
    shapely.box(*json.loads(fragment.metadata.metadata[b"geo"])["columns"]["geometry"]["bbox"])
    for fragment in dataset.get_fragments()
]
file_bounds = geopandas.GeoSeries(file_bounds, crs="EPSG:4326")

In [5]:
file_bounds

0      POLYGON ((-2.82298 -84.2946, -2.82298 -22.4999...
1      POLYGON ((-56.24995 -33.75032, -56.24995 -28.1...
2      POLYGON ((-50.62491 -28.82788, -50.62491 -22.4...
3      POLYGON ((-46.40526 -25.3127, -46.40526 -22.49...
4      POLYGON ((-44.99992 -27.42209, -44.99992 -22.4...
                             ...                        
231    POLYGON ((45.00003 -29.5314, 45.00003 -11.2499...
232    POLYGON ((31.22361 -33.75032, 31.22361 -28.476...
233    POLYGON ((32.96367 -29.53126, 32.96367 -25.312...
234    POLYGON ((33.75006 -26.71894, 33.75006 -23.905...
235    POLYGON ((178.80905 -80.41724, 178.80905 -22.4...
Length: 236, dtype: geometry

In [6]:
lonboard.viz(file_bounds)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

Next, we can visualize the row-group-based spatial partitions:

In [7]:
row_group_bounds = []
for fragment in dataset.get_fragments():
    meta = fragment.metadata
    field_xmin = meta.schema.names.index("xmin")
    field_ymin = meta.schema.names.index("ymin")
    field_xmax = meta.schema.names.index("xmax")
    field_ymax = meta.schema.names.index("ymax")
    for i in range(meta.num_row_groups):
        row_group = meta.row_group(i)
        xmin = row_group.column(field_xmin).statistics.min
        ymin = row_group.column(field_ymin).statistics.min
        xmax = row_group.column(field_xmax).statistics.max
        ymax = row_group.column(field_ymax).statistics.max
        row_group_bounds.append(shapely.box(xmin, ymin, xmax, ymax))

row_group_bounds = geopandas.GeoSeries(row_group_bounds, crs="EPSG:4326")

In [8]:
row_group_bounds

0        POLYGON ((-2.82298 -84.29461, -2.82298 -45, -1...
1        POLYGON ((-67.69156 -54.84377, -67.69156 -51.3...
2        POLYGON ((-68.67027 -53.4351, -68.67027 -47.81...
3        POLYGON ((-67.49979 -50.6241, -67.49979 -46.40...
4        POLYGON ((-67.49977 -47.10915, -67.49977 -45.0...
                               ...                        
16865    POLYGON ((47.77608 -23.55472, 47.77608 -23.027...
16866    POLYGON ((168.7505 -80.41725, 168.7505 -22.499...
16867    POLYGON ((170.15379 -50.54442, 170.15379 -45.7...
16868    POLYGON ((171.08829 -46.23061, 171.08829 -44.9...
16869    POLYGON ((178.80908 -77.85104, 178.80908 -45.8...
Length: 16870, dtype: geometry

In [9]:
lonboard.viz(row_group_bounds)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

## Get started with providing GeoParquet datasets

See the "Best Practices for Distributing GeoParquet" (https://github.com/opengeospatial/geoparquet/blob/main/format-specs/distributing-geoparquet.md) for several practical recommendations on how to distribute GeoParquet datasets (use good compression, use spatial partitioning with multiple files and with the `bbox` column).

## [extra] Parquet native Geometry/Geography logical types

While the current GeoParquet specification makes use of "binary" columns in the Parquet file, early 2025 the Parquet specification adopted geospatial types as native "logical" types (still using a binary storage of WKB under the hood). See https://cloudnativegeo.org/blog/2025/02/geoparquet-2.0-going-native/ for more context. 

In the future we expect that those new native types will be used, but while support is being implemented in many tools, for now it is recommended to use the GeoParquet 1.1 specification for widespread compatibility.

Small example using the data from https://geoarrow.org/data.html (download [natural-earth_cities.parquet](https://raw.githubusercontent.com/geoarrow/geoarrow-data/v0.2.0/natural-earth/files/natural-earth_cities.parquet) to run the following example):

In [1]:
import pyarrow.parquet as pq

In [2]:
f = pq.ParquetFile("natural-earth_cities.parquet")

In [3]:
f.metadata.schema

required group field_id=-1 schema {
  optional binary field_id=-1 name (String);
  optional binary field_id=-1 geometry (Geometry(crs=));
}

In [4]:
import geoarrow.pyarrow

In [5]:
pq.read_table("natural-earth_cities.parquet").schema

name: string
geometry: extension<geoarrow.wkb<WkbType>>